# Association_Rule 關聯規則
- 以下使用python3.7
- 使用mlxtend套件 pip install mlxtend

In [3]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

自定義一份資料集

In [4]:
data = {'ID':[1,2,3,4,5,6],
       'Onion':[1,0,0,1,1,1],
       'Potato':[1,1,0,1,1,1],
       'Burger':[1,1,0,0,1,1],
       'Milk':[0,1,1,1,0,1],
       'Beer':[0,0,1,0,1,0]}

In [5]:
df = pd.DataFrame(data)

In [6]:
df = df[['ID', 'Onion', 'Potato', 'Burger', 'Milk', 'Beer' ]]

In [7]:
df

,ID,Onion,Potato,Burger,Milk,Beer
0,1,1,1,1,0,0
1,2,0,1,1,1,0
2,3,0,0,0,1,1
3,4,1,1,0,1,0
4,5,1,1,1,0,1
5,6,1,1,1,1,0


### 設置支持度來篩選頻繁項集
- apriori(df, min_support=0.5, use_colnames=True)
- 上面選擇最小支持度為50%，實務上會根據樣本量及資料特性設定

In [10]:
frequent_itemsets = apriori(df[['Onion', 'Potato', 'Burger', 'Milk', 'Beer' ]], min_support=0.50, use_colnames=True)

In [11]:
frequent_itemsets

,support,itemsets
0,0.666667,(Onion)
1,0.833333,(Potato)
2,0.666667,(Burger)
3,0.666667,(Milk)
4,0.666667,"(Onion, Potato)"
5,0.500000,"(Onion, Burger)"
6,0.666667,"(Potato, Burger)"
7,0.500000,"(Potato, Milk)"
8,0.500000,"(Onion, Burger, Potato)"


### 計算規則
- association_rules(df, metric='lift', min_threshold=1)
- 可以指定不同的衡量標准與最小閥值

In [13]:
rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1)

In [14]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Onion),(Potato),0.666667,0.833333,0.666667,1.00,1.200,0.111111,inf
1,(Potato),(Onion),0.833333,0.666667,0.666667,0.80,1.200,0.111111,1.666667
2,(Burger),(Onion),0.666667,0.666667,0.500000,0.75,1.125,0.055556,1.333333
3,(Onion),(Burger),0.666667,0.666667,0.500000,0.75,1.125,0.055556,1.333333
4,(Burger),(Potato),0.666667,0.833333,0.666667,1.00,1.200,0.111111,inf
5,(Potato),(Burger),0.833333,0.666667,0.666667,0.80,1.200,0.111111,1.666667
6,"(Burger, Onion)",(Potato),0.500000,0.833333,0.500000,1.00,1.200,0.083333,inf
7,"(Burger, Potato)",(Onion),0.666667,0.666667,0.500000,0.75,1.125,0.055556,1.333333
8,"(Onion, Potato)",(Burger),0.666667,0.666667,0.500000,0.75,1.125,0.055556,1.333333
9,(Burger),"(Onion, Potato)",0.666667,0.666667,0.500000,0.75,1.125,0.055556,1.333333


返回的是各個的指標的數值，可以按照感興趣的指標排序觀察,但具體解釋還得參考實際數據的含義。

In [15]:
rules[(rules['lift'] >1.125) & (rules['confidence']> 0.8)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Onion),(Potato),0.666667,0.833333,0.666667,1.0,1.2,0.111111,inf
4,(Burger),(Potato),0.666667,0.833333,0.666667,1.0,1.2,0.111111,inf
6,"(Burger, Onion)",(Potato),0.500000,0.833333,0.500000,1.0,1.2,0.083333,inf


得到幾條比較有價值的結果：
- (洋蔥和馬鈴薯)、(漢堡和馬鈴薯)可以搭配著來賣
- 如果洋蔥和漢堡都在購物籃中, 顧客買馬鈴薯的可能性也滿高的，如果他籃子裡面沒有，可以推薦一下.

### one-hot-encoding 編碼轉換
- 實務資料通常需要轉換成 one hot encoding
- 下面以逗號分隔的文字資料，為範例演練

In [17]:
retail_shopping_basket = {'ID':[1,2,3,4,5,6],
                         'Basket':[['Beer', 'Diaper', 'Pretzels', 'Chips', 'Aspirin'],
                                   ['Diaper', 'Beer', 'Chips', 'Lotion', 'Juice', 'BabyFood', 'Milk'],
                                   ['Soda', 'Chips', 'Milk'],
                                   ['Soup', 'Beer', 'Diaper', 'Milk', 'IceCream'],
                                   ['Soda', 'Coffee', 'Milk', 'Bread'],
                                   ['Beer', 'Chips']
                                  ]
                         }

In [18]:
retail = pd.DataFrame(retail_shopping_basket)

In [19]:
retail

,ID,Basket
0,1,"[Beer, Diaper, Pretzels, Chips, Aspirin]"
1,2,"[Diaper, Beer, Chips, Lotion, Juice, BabyFood,..."
2,3,"[Soda, Chips, Milk]"
3,4,"[Soup, Beer, Diaper, Milk, IceCream]"
4,5,"[Soda, Coffee, Milk, Bread]"
5,6,"[Beer, Chips]"


In [20]:
# 先把ID欄位抽出來
retail_id = retail.drop('Basket' ,1)
retail_id

,ID
0,1
1,2
2,3
3,4
4,5
5,6


In [21]:
# 用逗號分隔每個字詞
retail_Basket = retail.Basket.str.join(',')
retail_Basket

0              Beer,Diaper,Pretzels,Chips,Aspirin
1    Diaper,Beer,Chips,Lotion,Juice,BabyFood,Milk
2                                 Soda,Chips,Milk
3                  Soup,Beer,Diaper,Milk,IceCream
4                          Soda,Coffee,Milk,Bread
5                                      Beer,Chips
Name: Basket, dtype: object

In [22]:
# 轉one-hot
retail_Basket = retail_Basket.str.get_dummies(',')
retail_Basket

,Aspirin,BabyFood,Beer,Bread,Chips,Coffee,Diaper,IceCream,Juice,Lotion,Milk,Pretzels,Soda,Soup
0,1,0,1,0,1,0,1,0,0,0,0,1,0,0
1,0,1,1,0,1,0,1,0,1,1,1,0,0,0
2,0,0,0,0,1,0,0,0,0,0,1,0,1,0
3,0,0,1,0,0,0,1,1,0,0,1,0,0,1
4,0,0,0,1,0,1,0,0,0,0,1,0,1,0
5,0,0,1,0,1,0,0,0,0,0,0,0,0,0


In [23]:
# 把ID欄位組回來
retail = retail_id.join(retail_Basket)
retail

,ID,Aspirin,BabyFood,Beer,Bread,Chips,Coffee,Diaper,IceCream,Juice,Lotion,Milk,Pretzels,Soda,Soup
0,1,1,0,1,0,1,0,1,0,0,0,0,1,0,0
1,2,0,1,1,0,1,0,1,0,1,1,1,0,0,0
2,3,0,0,0,0,1,0,0,0,0,0,1,0,1,0
3,4,0,0,1,0,0,0,1,1,0,0,1,0,0,1
4,5,0,0,0,1,0,1,0,0,0,0,1,0,1,0
5,6,0,0,1,0,1,0,0,0,0,0,0,0,0,0


In [24]:
frequent_itemsets_2 = apriori(retail.drop('ID',1), use_colnames=True)

In [25]:
frequent_itemsets_2

,support,itemsets
0,0.666667,(Beer)
1,0.666667,(Chips)
2,0.500000,(Diaper)
3,0.666667,(Milk)
4,0.500000,"(Beer, Chips)"
5,0.500000,"(Diaper, Beer)"


如果光看支持度support(X>Y)，(Beer, Chips)和(Beer, Diaper)都是很頻繁的，因此要更進步分析才能比較差異。

In [26]:
association_rules(frequent_itemsets_2, metric='lift')

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Beer),(Chips),0.666667,0.666667,0.5,0.75,1.125,0.055556,1.333333
1,(Chips),(Beer),0.666667,0.666667,0.5,0.75,1.125,0.055556,1.333333
2,(Diaper),(Beer),0.500000,0.666667,0.5,1.00,1.500,0.166667,inf
3,(Beer),(Diaper),0.666667,0.500000,0.5,0.75,1.500,0.166667,2.000000


從lift可以發現，(Diaper, Beer)更相關一些。

### 實戰演練-電影題材關聯
資料集：[MovieLens (small)](https://grouplens.org/datasets/movielens/)

In [27]:
movies = pd.read_csv('movies.csv')

In [28]:
movies.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


資料中包括電影名字與電影類型的標籤，第一步還是先轉換成one-hot。

In [29]:
movies_ohe = movies.drop('genres',1).join(movies.genres.str.get_dummies())

In [30]:
movies_ohe.head()

,movieId,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
movies_ohe.shape

(9125, 22)

資料集包括9125部電影，一共有20種不同類型。

In [32]:
movies_ohe.set_index(['movieId','title'],inplace=True)

In [33]:
movies_ohe.head()

,,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,title,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,Jumanji (1995),0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,Grumpier Old Men (1995),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,Waiting to Exhale (1995),0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
5,Father of the Bride Part II (1995),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [34]:
# 實務上min_support會根據樣本數量、項集數量去預估，通常不會設定太高，不然會跑不出結果
frequent_itemsets_movies = apriori(movies_ohe,use_colnames=True, min_support=0.025)

In [35]:
frequent_itemsets_movies.head()

,support,itemsets
0,0.169315,(Action)
1,0.122411,(Adventure)
2,0.048986,(Animation)
3,0.063890,(Children)
4,0.363288,(Comedy)


In [36]:
rules_movies =  association_rules(frequent_itemsets_movies, metric='lift', min_threshold=1.25)

In [37]:
rules_movies.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Action),(Adventure),0.169315,0.122411,0.058301,0.344337,2.812955,0.037575,1.338475
1,(Adventure),(Action),0.122411,0.169315,0.058301,0.476276,2.812955,0.037575,1.586111
2,(Action),(Crime),0.169315,0.120548,0.038247,0.225890,1.873860,0.017836,1.136081
3,(Crime),(Action),0.120548,0.169315,0.038247,0.317273,1.873860,0.017836,1.216716
4,(Sci-Fi),(Action),0.086795,0.169315,0.040986,0.472222,2.789015,0.026291,1.573929


In [38]:
# 找出給小孩題材相關電影
rules_movies[(rules_movies.antecedents=={'Children'})].sort_values(by=['lift'], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
15,(Children),(Animation),0.06389,0.048986,0.027068,0.423671,8.648758,0.023939,1.650122
8,(Children),(Adventure),0.06389,0.122411,0.029260,0.457976,3.741299,0.021439,1.619096
17,(Children),(Comedy),0.06389,0.363288,0.032877,0.514580,1.416453,0.009666,1.311672


Children和Animation 這兩個題材是最相關的了，其實滿符合現實常識。再來是冒險類型，相關性高也滿好理解。最後一項是Comedy喜劇，雖然有相關，但跟其他比起來相關性就沒這麼高。